In [1]:
#import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt

##### The input image

In [2]:
# Load the input image
input_image = cv2.imread('.\\originals\\ANLM_only.jpeg', cv2.IMREAD_GRAYSCALE)

In [3]:
r, c = input_image.shape
print(r, c)

1024 512


##### Helper functions

In [4]:
### display image ###
def display(img1):
   plt.imshow(img1, cmap='gray')
   plt.axis('off')  # Turn off axis labels and ticks
   plt.show()

In [ ]:
#extract the patches from the image
    # returns the list of patches
def get_patches(img, box):
    # Get the dimensions of the input image
    rows, cols = img.shape
    box_width, box_height = box[1], box[0]

    # Calculate the number of patches in each dimension
    num_patches_x = cols // box_width
    num_patches_y = rows // box_height

    # Create a list to store the patches
    patches = []

    # Iterate through the patches and create a list of non-overlapping patches
    for i in range(num_patches_y):
        for j in range(num_patches_x):
            # Define the coordinates of the current patch
            patch_x1 = j * box_width
            patch_x2 = (j + 1) * box_width
            patch_y1 = i * box_height
            patch_y2 = (i + 1) * box_height

            # Extract the patch using slicing
            patch = img[patch_y1:patch_y2, patch_x1:patch_x2]

            # Append the patch to the list
            patches.append(patch)

    return patches

##### Image processing

In [3]:
############# contrast enhancement ##################
def denoise(img, threshold=50, box=(3,5)):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = img.copy()

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # denoise the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = patch[patch <= threshold]*(0.8)
                # make the non-black pixels brighter
                patch[patch > threshold] = avg*(1.2)
            #else:
                # make all pixels avg
                #patch[:, :] = avg*(1.1)

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

In [4]:
######### blurring #################

#Gaussian blurr
def blurr(img, kernel=(7,7)):
    denoised_image = cv2.GaussianBlur(img, kernel, 0) #apply Gaus. blur
    return denoised_image

##### Functions

In [ ]:
### image processing ###
def process(img, threshold, box):
    h, w= box
    result_img = np.copy(img)
    half_w, half_h = w // 2, h // 2

    for y in range(half_h, img.shape[0] - half_h):
        for x in range(half_w, img.shape[1] - half_w):
            patch = img[y - half_h:y + half_h + 1, x - half_w:x + half_w + 1]
            patch_avg = np.mean(patch)

            if img[y, x] < threshold:
                continue
            else:
                result_img[y, x] = patch_avg

    return result_img

In [79]:
#replace non-black pixels with patch average value
def process2(img, threshold, box):
    # Get the dimensions of the input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = np.zeros((rows, cols))

    # Define the dimensions of the box (patch)
    box_width, box_height = box[1], box[0]

    # process the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate patch average
            avg = np.mean(patch[:,:])

            # Chnage all non-black pixels to avg
            patch[patch > threshold] = avg

            # Update the patch in the result image 
            result[y:y+box_height, x:x+box_width] = patch

    return result

In [5]:
# make black pixels blacker (root n)
# replace non black pixels by average patch value
def process3(img, threshold, box):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = np.zeros((rows, cols))

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # process the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = avg*(0.75)
                # make the non-black pixels brighter
                patch[patch > threshold] = avg*(1.5)
            else:
                # make all pixels avg
                patch[:, :] = avg*(1.1)

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

In [81]:
# make black pixels blacker (root n)
# replace non black pixels by average patch value
def process4(img, threshold, box):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = np.zeros((rows, cols))

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # process the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = patch[patch <= threshold]*(0.5)
                # make the non-black pixels brighter
                patch[patch > threshold] = patch[patch > threshold]*(1.5)
            else:
                # make all pixels avg
                patch[:, :] = avg*(1.1)

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

##### Testing

In [8]:
##### ANLM only #####
# Load the input image
input_image = cv2.imread('.\\originals\\ANLM_only.jpeg', cv2.IMREAD_GRAYSCALE)


# hyper-parameters
threshold_value = 25
box_size = (3,4) #(rows, cols)

# Process the image
result_image = process3(input_image, threshold_value, box_size)
#temp = blurr(temp, (5,5))
#result_image = denoise(temp, threshold=30, box=(5,5)) 

#save the output
cv2.imwrite('output.jpg', result_image)

True

In [12]:
##### ANLM only #####
# Load the input image
path = "C:\\Users\\Samruddhi\\Desktop\\Internship 2\\Code\\2. Enhancing retina details\\test\\Dolvin_Mac_OS-228.jpg"
input_image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

# hyper-parameters
threshold_value = 20
box_size = (3,5) #(rows, cols)

# Process the image
result_image = process3(input_image, threshold_value, box_size)

#save the output
cv2.imwrite('test_output.jpg', result_image)

True

In [92]:
###### ANLM and CE #############

test = cv2.imread('.\\originals\\ANLM_CE.png', cv2.IMREAD_GRAYSCALE)

# hyper-parameters
threshold_value = 25
box_size = (3,5) #(rows, cols)

# Process the image
temp = process3(test, threshold_value, box_size)
result = blurr(temp, (7,7))

#save the image
cv2.imwrite('blurred.jpg', result)

True